In [25]:
# Solve a generalized assignment problem between the different tracklets 
# setting similarity weights on the assignments such that we maximize the
# total assignment weigthed objective and match the most likely tracklets
from gurobipy import *
from numpy import *
import os
import fnmatch

In [90]:
print cwd + '/3_obj_less_noisy'

/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj_less_noisy


In [152]:
import fnmatch
import os
cwd = os.getcwd()
matches = []
for root, dirnames, filenames in os.walk(cwd + '/3_obj'):
    for filename in fnmatch.filter(filenames, 'Pij_2.csv'):
        file_path = os.path.join(root, filename)
        P = loadtxt(open(root + '/' + filename, "rb"), delimiter=",")
#         if root == '/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj_obsolete/csv_3_1':
        print root
        print P
        numT = shape(P)[0]
        try:
            numC = shape(P)[1]
        except:
            numC = 0;    

        print numC
        T=range(numT)
        C=range(numC)

        # Initialize the model
        m=Model("Assignment")

        # Define the binary variables Xs
        X = []
        for t in T:
            X.append([])
            if numC != 0:
                for c in C:
                    X[t].append(m.addVar(vtype=GRB.BINARY,name="X%d%d"% (t, c)))
            else:
                X[t].append(m.addVar(vtype=GRB.BINARY,name="X%d%d"% (t, 0)))

        # Set the optimization direction
        m.update()
        m.modelSense = GRB.MAXIMIZE


        # Define the assignment constraints
        constraintT = []
        constraintC = []


        if numC != 0:
            # One variable assigned per column
            for t in range(numT):
                constraintT.append(m.addConstr(quicksum(X[t][c] for c in range(numC))
                                               <= 1 ,'constraintT%d' % t))
            # One variable assigned per row
            for c in range(numC):
                constraintT.append(m.addConstr(quicksum(X[t][c] for t in range(numT))
                                               <= 1 ,'constraintC%d' % t))

            # Define objective
            m.setObjective(quicksum(quicksum([X[t][c]*P[t][c] 
                                    for c in range(numC)]) 
                                    for t in range(numT)))
        else:
            # One variable assigned per column
            for t in range(numT):
                constraintT.append(m.addConstr(X[t][0]<= 1 ,'constraintT%d' % t))  

            # One variable assigned per row
            constraintT.append(m.addConstr(quicksum(X[t][0] for t in range(numT))
                                               <= 1 ,'constraintC%d' % t))

            # Define objective                 
            m.setObjective(quicksum(X[t][0]*P[t] for t in range(numT)))



        m.update()

        # Solve the optimization problem
        m.optimize()

        # Print runtime and solution

        print 'runtime is',m.Runtime, 's'
        print m


        if numC != 0:
            Xsol = zeros((numT,numC))
        else:
            Xsol = zeros(numT)
        # Print solution
        if m.status == GRB.Status.OPTIMAL:
            for t in range(numT):
                if numC != 0:
                    for c in range(numC):
                        if X[t][c].x > 0.5:
                            print('%g -> %g' % (t, c))
                            Xsol[t][c] = 1
                else:
                    if X[t][0].x > 0.5:
                        print('0 -> %g' % (t))
                        Xsol[t] = 1
        print Xsol
        name_sol = root + "/Assignment.csv"
        if numC != 0:
            savetxt(name_sol, Xsol, delimiter=",")
        else:
            savetxt(name_sol, Xsol[None], delimiter=",")
        

/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_65
[[ -9.99410000e+02  -9.99410000e+02  -9.99410000e+02   3.80360000e-02
    1.00000000e-02]
 [ -9.99410000e+02  -9.99410000e+02  -9.99410000e+02   1.21420000e-01
    3.08200000e-03]]
5
Optimize a model with 7 rows, 10 columns and 20 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.041118
Presolve removed 3 rows and 6 columns
Presolve time: 0.00s
Presolved: 4 rows, 4 columns, 8 nonzeros
Variable types: 0 continuous, 4 integer (4 binary)

Root relaxation: objective 1.314200e-01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.1314200    

Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.052484
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 0.13849 0.052484 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.384900000000e-01, best bound 1.384900000000e-01, gap 0.0000%
runtime is 0.0265231132507 s
<gurobi.Model MIP instance Assignment: 7 constrs, 10 vars, Parameter changes: LogFile=gurobi.log>
0 -> 4
1 -> 3
[[ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_85
[[ -9.99560000e+02  -9.99560000e+02  -9.99560000e+02   1.00000000e-02
    2.55830000e-02]
 [ -9.99560000e+02 

runtime is 0.0264229774475 s
<gurobi.Model MIP instance Assignment: 7 constrs, 10 vars, Parameter changes: LogFile=gurobi.log>
0 -> 3
[[ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_45
[[ -9.99600000e+02  -9.99600000e+02  -9.99600000e+02   1.78890000e-03
    1.00000000e-02]
 [ -9.99600000e+02  -9.99600000e+02  -9.99600000e+02   2.98730000e-02
   -9.40650000e-02]]
5
Optimize a model with 7 rows, 10 columns and 20 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.0017889
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 0.039873 0.00178

Found heuristic solution: objective 0.030522

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.030522 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.052200000000e-02, best bound 3.052200000000e-02, gap 0.0000%
runtime is 0.0169818401337 s
<gurobi.Model MIP instance Assignment: 7 constrs, 10 vars, Parameter changes: LogFile=gurobi.log>
0 -> 4
1 -> 3
[[ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_98
[[ -9.99730000e+02  -9.99730000e+02  -9.99730000e+02   1.00000000e-02
    1.60080000e-01]
 [ -9.99730000e+02  -9.99730000e+02  -9.99730000e+02  -5.50320000e-03
    1.59380000e-01]]
5
Optimize a model with 7 rows, 10 columns and 20 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS rang

5
Optimize a model with 8 rows, 15 columns and 30 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.22015
Presolve removed 8 rows and 15 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.22015 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.201500000000e-01, best bound 2.201500000000e-01, gap 0.0000%
runtime is 0.0177450180054 s
<gurobi.Model MIP instance Assignment: 8 constrs, 15 vars, Parameter changes: LogFile=gurobi.log>
0 -> 2
[[ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_31
[[ -9.99770000e+02  -9.99770000e+02  -9.9977

...

Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.036019

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.036019 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.601900000000e-02, best bound 3.601900000000e-02, gap 0.0000%
runtime is 0.0162129402161 s
<gurobi.Model MIP instance Assignment: 7 constrs, 10 vars, Parameter changes: LogFile=gurobi.log>
0 -> 4
1 -> 3
[[ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_34
[[ -9.99450000e+02  -9.99450000e+02  -9.99450000e+02   1.00000000e-02
    1.87400000e-01]
 [ -9.99450000e+02  -9.99450000e+02  -9.99450000e+02  -9.99450000e+02
    1.83330000e-02]]
5
Optimize a model with 7 rows, 10 

Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.058503
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.058503 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.850300000000e-02, best bound 5.850300000000e-02, gap 0.0000%
runtime is 0.0160391330719 s
<gurobi.Model MIP instance Assignment: 7 constrs, 10 vars, Parameter changes: LogFile=gurobi.log>
0 -> 4
1 -> 3
[[ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_15
[[ -9.98530000e+02   1.00000000e-02  -9.98530000e+02   6.35870000e-01
   -9.98530000e+02   6.94850000e-01   7.90800000e-01]
 [ -9.98530000e+02  -9.98530000e+02  -9.985300

  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.052157

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.052157 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.215700000000e-02, best bound 5.215700000000e-02, gap 0.0000%
runtime is 0.0215601921082 s
<gurobi.Model MIP instance Assignment: 7 constrs, 10 vars, Parameter changes: LogFile=gurobi.log>
0 -> 3
1 -> 4
[[ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_61
[[ -9.99830000e+02  -9.99830000e+02  -9.99830000e+02   1.10160000e-01
    1.00000000e-02]
 [ -9.99830000e+02  -9.99830000e+02  -9.99830000e+02   7.98930000e-03
   -1.25680000e-02]]
5
Optimize a model with 7 rows, 10 columns and 20 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)


runtime is 0.0193810462952 s
<gurobi.Model MIP instance Assignment: 7 constrs, 10 vars, Parameter changes: LogFile=gurobi.log>
0 -> 3
1 -> 4
[[ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_17
[[ -9.99680000e+02  -9.99680000e+02  -9.99680000e+02   8.92800000e-02
    1.00000000e-02]
 [ -9.99680000e+02  -9.99680000e+02  -9.99680000e+02   1.59660000e-01
   -5.85210000e-02]]
5
Optimize a model with 7 rows, 10 columns and 20 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.08928
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 0.16966 0.0

/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_7
[[ -9.99710000e+02  -9.99710000e+02  -9.99710000e+02  -9.99710000e+02
   -6.88720000e-02  -1.37850000e-01]
 [ -9.99710000e+02  -9.99710000e+02  -9.99710000e+02  -9.99710000e+02
    6.22370000e-02  -1.77730000e-01]
 [ -9.99710000e+02  -9.99710000e+02  -9.99710000e+02  -9.99710000e+02
   -9.99710000e+02   1.00000000e-02]]
6
Optimize a model with 9 rows, 18 columns and 36 nonzeros
Variable types: 0 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.072237

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.072237 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.223700000000e-02, best bound 7.223700000000e-02, gap 0.0000%
runtime is 0.01441001892

Optimize a model with 10 rows, 25 columns and 50 nonzeros
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: -0 

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%


runtime is 0.0195119380951 s
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_22
